<div style="background-color:rgba(78, 188, 130, 0.05); text-align:center; vertical-align: middle; padding:20px 0;border:3px; border-style:solid; padding: 0.5em; border-color: rgba(78, 188, 130, 1.0); color: #000000;">

<img src="figs/qr_logo.png" width="700"/>

<h1><strong>Quantum Summer School</strong></h1>

<h2><strong>Episode 13</strong></h2>

<h3><strong>Quantum Error Correction</strong></h3>

</div>

*In this session, we will learn about quantum error correction!*

<div style="background-color:rgba(255, 248, 240, 1.0); text-align:left; vertical-align: middle; padding:20px 0;border:3px; border-style:solid; padding: 0.5em; border-color: rgba(255, 142, 0, 1.0); color: #000000;">

## Objectives
1. Explore 3 qubit bit & phase flip codes
2. Learn the Shor Code

<div/>

## Setup & Imports

In [ ]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2,  QrStatevector as Statevector, QrSamplerV2
from quantumrings.toolkit.qiskit import QrEstimatorV2 as Estimator
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister, transpile
!pip -q install qiskit_aer
from qiskit_aer import Aer
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import Statevector, PauliList, Pauli
import matplotlib.pyplot as plt
import numpy as np

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
    mixed_unitary_error,
    coherent_unitary_error
)

provider = QuantumRingsProvider()
backend = QrBackendV2(provider, num_qubits=10)

# Helpers

Flip the i-th bit of a binary string s. Index i is 0-based. Example: flip_bit("001111", 2) -> "000111"

In [ ]:
def flip_bit(s: str, i: int) -> str:
    if not (0 <= i < len(s)):
        raise ValueError("Index out of range")

    # flip the character
    flipped = '0' if s[i] == '1' else '1'
    return s[:i] + flipped + s[i+1:]

# 1. 3-Qubit Bit-Flip Code

<div style="background-color:rgba(255, 245, 253, 1.0); text-align:left; vertical-align: middle; padding:20px 0;border:3px; border-style:solid; padding: 0.5em; border-color: rgba(255, 142, 235, 1.0); color: #000000;">

One of the main challenges standing in the way of progress in quantum computing is the fact that quantum hardware is noisy! Interactions with the environment decohere our qubits, leading to bit and phase-flip errors. There are all kinds of other noise channels as well. For comparison, a classical computer has an error rate of 1 in $10^{18}$! In all your time using a regular computer or smart phone, you have never witnessed a bit flip error impacting the performance of the device. Our best quantum computers, however, have an error rate of around 1-0.1%, many orders of magnitude higher than a classical computer.

Error mitigation techniques, like we saw in episode 10, can help, but they aren't enough. In order to have fault-tolerant quantum computing, we need to create logical qubits which are protected from noise. The idea is to take many physical qubits and encode the logical state into a highly entangled state across the physical qubits. Then, we will be able to detect when an error has occured and correct for it.

The simplest example of a quantum error-correcting (QEC) code is the three qubit bit flip code. This code mimics a classical repetition code, where we encode the logical bit $\overline{0}$ in many physical bits $00\dots 0$, and the logical bit $\overline{1}$ in many physical bits $11\dots 1$. If one bit flips, we can easily detect that and correct for it. With n=5 physical bits, we can detect and correct up to $\lfloor \frac{n-1}{2} \rfloor$ errors. Ex: if $n=5$, then we can detect/correct up to 2 errors. If there were 3 errors, for instance $00000 \rightarrow 11100$, we would incur a logical error, because we would assume by majority that the correct logical state is $\overline{1}$.

For the three qubit bit flip code, we will be able to correct one bit-flip error. There are four steps to the protocol:
1. **Encode the logical state onto physical qubits.** We define $|\overline{0}\rangle = |000\rangle$ and $|\overline{1}\rangle = |111\rangle$. If our logical state is $|\psi \rangle = \alpha|\overline{0}\rangle + \beta |\overline{1}\rangle$, then we prepare one qubit in that state and use CNOT gates to spread the state onto the other two qubits.
2. **Apply logical operations.** Here, we can perform the desired operations on our logical state. If we want to apply an H gate to our logical state, we would apply an H gate to each of the three qubits. In this part, our qubits can also experience noise (such as bit flip errors).
3. **Detect errors.** We use parity measurements to detect if an error has occured. We use two ancilla qubits, one to measure the parity of qubits 0 and 1 and one to measure the parity of qubits 1 and 2. Recall that a parity measurement should give us 0 if both qubits are in the same state and 1 if they are in different states. The result of our parity measurements is called the error syndrome.
4. **Correct the state based on the error syndrome.** Each error syndrome corresponds to a distinct single qubit bit-flip error. Based on the syndrome, apply an X gate as needed to repair the state.

Let's explore this repetition code together.

<div/>

In [ ]:
qr = QuantumRegister(5, "q")
cr = ClassicalRegister(5, "c")
qc = QuantumCircuit(qr,cr, name='QSS13.1_bitflip')

# prepare state
#

# encode
#
#

# gates / noise
#

qc.barrier()

# detect
#
#
#
#

qc.barrier()

# measure
qc.measure(qr[3:],cr[3:])

qc.barrier()

# decode
qc.measure(qr[0:3],cr[0:3])
qc.draw()

In [ ]:
job = backend.run(qc, shots=1000)
result = job.result()
counts = result.get_counts()

decoded_counts = {}
for string, num_counts in counts.items():
    syndrome = string[:2]
    data_string = string[2:]   # the logical qubit measurement

    # apply the correction rule in software
    if syndrome == "00":
        corrected = data_string
    elif syndrome == "10":
        corrected = ##
    elif syndrome == "11":
        corrected = ##
    else:
        corrected = ##

    decoded_counts[corrected] = decoded_counts.get(corrected, 0) + num_counts

plot_histogram(decoded_counts)

**Question:** We are performing a measurement in the middle of our circuit! (the parity measurements) We know that measurement collapses our qubit states. How come we are still able to preserve the logical state at the end of the circuit?

## A More Compact Version

We can condense the QEC code above by absorbing the parity measurements and correction into a pair of CNOTs and a Toffoli gate. In the end, the logical state will be stored only on qubit 0. Let's see how it works.

In [ ]:
qr = QuantumRegister(3, "q")
cr = ClassicalRegister(1, "c")
qc = QuantumCircuit(qr,cr, name='QSS13.2_compact')

# prepare state
# 

# encode
qc.cx(0,1)
qc.cx(0,2)

qc.barrier()

# gates / noise
#

qc.barrier()

# detect/correct
qc.cx(0,1)
qc.cx(0,2)
qc.ccx(1,2,0)

# decode
qc.measure(0,cr)
qc.draw()

In [ ]:
job = backend.run(qc, shots=1000)
result = job.result()
counts = result.get_counts()

plot_histogram(counts)

<div style="background-color:rgba(252, 245, 255, 1.0); text-align:left; vertical-align: middle; padding:20px 0;border:3px; border-style:solid; padding: 0.5em; border-color: rgba(190, 111, 227, 1.0); color: #000000;">

### Challenge Problem: 

**A)** What are the weaknesses of the QEC code above? What would you want a QEC code to be able to do to be really useful?

**B)** What would you change in the circuit above so that the logical state is protected against a one-qubit phase flip? 

*Hint: the general structure should be the same, but we want to change basis!*

<div/>

In [ ]:
def encode(flip_type='bit', barriers=True):

    qr = QuantumRegister(3)
    qc = QuantumCircuit(qr, name='QSS13.3_encode')
    
    qc.cx(0,1)
    qc.cx(0,2)

    if flip_type == 'phase':
        qc.h(qr)

    if barriers:
        qc.barrier()

    return qc

def correct(flip_type='bit', barriers=True):

    qr = QuantumRegister(3)
    qc = QuantumCircuit(qr, name='QSS13.4_correct')

    if barriers:
        qc.barrier()

    if flip_type == 'phase':
        qc.h(qr)
    
    qc.cx(0,1)
    qc.cx(0,2)

    qc.ccx(1,2,0)

    if barriers:
        qc.barrier()

    return qc

In the part where we would normally apply a logical operation, we can put Identity gates and then run the circuit using Qiskit's Aer Simulator for a noisy backend.

In [ ]:
qr = QuantumRegister(3)
cr = ClassicalRegister(1)
qc = QuantumCircuit(qr,cr, name='QSS13.5_noisy')

# prepare state
#

# encode state
qc.compose(encode(flip_type='bit'), inplace=True)

# inject noise
qc.id(qr)

# correct errors
qc.compose(correct(flip_type='bit'), inplace=True)

qc.measure(0,cr)

qc.draw()

## Using Qiskit Aer (for Noisy Backend)

In [ ]:
p = 0.05  # probability of bit-flip
bit_flip_error = pauli_error([('X', p), ('I', 1 - p)])

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(bit_flip_error, ['id'])

# Create noisy simulator backend
sim_noise = AerSimulator(noise_model=noise_model)
     
# Transpile circuit for noisy basis gates
passmanager = generate_preset_pass_manager(backend=sim_noise, optimization_level=0) # make sure optimization level is 0 to preserve id gates
qc_noisy = passmanager.run(qc)

# Run and get counts
result = sim_noise.run(qc_noisy, shots=1000).result()
counts = result.get_counts()
plot_histogram(counts)

**Question:** How does the logical error rate compare to the physical error rate?

## 3-Qubit Phase-Flip Code

We can now also play around with the phase-flip code.

In [ ]:
qr = QuantumRegister(3)
cr = ClassicalRegister(1)
qc = QuantumCircuit(qr,cr)

# prepare state
#

# encode state
qc.compose(encode(flip_type='phase'), inplace=True)

# inject noise
qc.id(qr)

# correct errors
qc.compose(correct(flip_type='phase'), inplace=True)

qc.measure(0,cr)

qc.draw()

In [ ]:
p = 0.1  # probability of phase-flip
phase_flip_error = pauli_error([('Z', p), ('I', 1 - p)])

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(phase_flip_error, ['id'])

# Create noisy simulator backend
sim_noise = AerSimulator(noise_model=noise_model)
     
# Transpile circuit for noisy basis gates
passmanager = generate_preset_pass_manager(backend=sim_noise, optimization_level=0)
qc_noisy = passmanager.run(qc)

# Run and get counts
result = sim_noise.run(qc_noisy, shots=1000).result()
counts = result.get_counts()
plot_histogram(counts)

<div style="background-color:rgba(252, 245, 255, 1.0); text-align:left; vertical-align: middle; padding:20px 0;border:3px; border-style:solid; padding: 0.5em; border-color: rgba(190, 111, 227, 1.0); color: #000000;">

### Challenge Problem: 

**A)** Why can't we detect X errors with the phase-flip code or Z errors with the bit-flip code? Can you think of any way to combine them to get the best of both worlds?

**B)** Can the bit-flip code detect Y errors? Can it correct Y errors? What about the phase-flip code?

<div/>

# 2. The Shor Code

<div style="background-color:rgba(247, 255, 245, 1.0); text-align:left; vertical-align: middle; padding:20px 0;border:3px; border-style:solid; padding: 0.5em; border-color: rgba(0, 153, 51, 1.0); color: #000000;">

The Shor Code combines the bit flip and phase flip code together to be able to correct any single qubit Pauli error. By a Pauli error, we mean any error which can be expressed as a linear combination of $\sigma_x, \sigma_y, \sigma_z$ and the identity.

The Shor code encodes the logical states as follows:
$$ |\overline{0}\rangle = \frac{1}{2\sqrt{2}} (|000\rangle + |111\rangle) \otimes (|000\rangle + |111\rangle) \otimes (|000\rangle + |111\rangle) $$
$$ |\overline{1}\rangle = \frac{1}{2\sqrt{2}} (|000\rangle - |111\rangle) \otimes (|000\rangle - |111\rangle) \otimes (|000\rangle - |111\rangle) $$

Practically speaking, this means that three bit flip repetition codes are sandwiched between one phase flip repetition code, using one qubit each from the bit flip repetition codes. Let's code it up together!

</div>

In [ ]:
qr = QuantumRegister(9)
cr = ClassicalRegister(1)
qc = QuantumCircuit(qr,cr, name='QSS13.6_shor')

# prepare state
# you can use 0,1 to probe Z errors and +,- to probe X errors

# encode state - phase
#

for i in [0,3,6]:
    # encode state - bit
    #

# inject noise
qc.id(qr)

for i in [0,3,6]:
    # correct errors - bit
    #

# correct errors - phase
#

qc.measure(0,cr)

qc.draw()

In [ ]:
p_bit_flip = 0
p_phase_flip = 0
bit_flip_error = pauli_error([('X', p_bit_flip), ('I', 1 - p_bit_flip)])
phase_flip_error = pauli_error([('Z', p_phase_flip), ('I', 1 - p_phase_flip)])

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(bit_flip_error, ['id'],)
noise_model.add_all_qubit_quantum_error(phase_flip_error, ['id'], warnings=False)

# Create noisy simulator backend
sim_noise = AerSimulator(noise_model=noise_model)
     
# Transpile circuit for noisy basis gates
passmanager = generate_preset_pass_manager(backend=sim_noise, optimization_level=0)
qc_noisy = passmanager.run(qc)

# Run and get counts
result = sim_noise.run(qc_noisy, shots=1000).result()
counts = result.get_counts()
plot_histogram(counts)

In [ ]:
def shor_code():
    qr = QuantumRegister(9)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr,cr, name='QSS13.7_shor_noise')

    # encode state - phase
    qc.compose(encode(flip_type='phase'), qubits=[0,3,6], inplace=True)
    
    for i in [0,3,6]:
        # encode state - bit
        qc.compose(encode(flip_type='bit'), qubits=[i,i+1,i+2], inplace=True)
    
    # inject noise
    qc.id(qr)
    
    for i in [0,3,6]:
        # correct errors - bit
        qc.compose(correct(flip_type='bit'), qubits=[i,i+1,i+2], inplace=True)
    
    # correct errors - phase
    qc.compose(correct(flip_type='phase'), qubits=[0,3,6], inplace=True)
    
    qc.measure(0,cr)
    
    return qc

In [ ]:
def run_shor_code_with_noise(p_bit_flip = 0, p_phase_flip = 0, shots = 1000):

    qc = shor_code()

    bit_flip_error = pauli_error([('X', p_bit_flip), ('I', 1 - p_bit_flip)])
    phase_flip_error = pauli_error([('Z', p_phase_flip), ('I', 1 - p_phase_flip)])
    
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(bit_flip_error, ['id'],)
    noise_model.add_all_qubit_quantum_error(phase_flip_error, ['id'], warnings=False)
    
    # Create noisy simulator backend
    sim_noise = AerSimulator(noise_model=noise_model)
         
    # Transpile circuit for noisy basis gates
    passmanager = generate_preset_pass_manager(backend=sim_noise, optimization_level=0)
    qc_noisy = passmanager.run(qc)
    
    # Run and get counts
    result = sim_noise.run(qc_noisy, shots=shots).result()
    counts = result.get_counts()

    error_rate = counts.get('1',0)/shots
    return error_rate

In [ ]:
p_phase_flip = np.linspace(0,0.1,15)

errors = np.full(len(p_phase_flip), np.nan)

for i, p_p in enumerate(p_phase_flip):

    errors[i] = run_shor_code_with_noise(p_bit_flip = 0, p_phase_flip = p_p, shots = 2000)

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(p_phase_flip, errors, "o-", label="logical error rate")
plt.plot(p_phase_flip, p_phase_flip, "s--", label="physical error rate")

plt.xlabel("Phase-flip probability")
plt.ylabel("Error rate")
plt.title("Logical vs Physical Error Rate")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

<div style="background-color:rgba(252, 245, 255, 1.0); text-align:left; vertical-align: middle; padding:20px 0;border:3px; border-style:solid; padding: 0.5em; border-color: rgba(190, 111, 227, 1.0); color: #000000;">

### Challenge Problem: 

**A)** Play around more with the Shor code. The Shor code can correct any single-qubit error. But it can also correct certain combinations of two or more qubit errors. What kind of multi-qubit errors can the Shor code correct?

**B)** Today we learned about the surface code as well. Try coding up the [[9,1,3]] surface code! Inject error into the circuit and see what error syndromes result. Can you find different combinations of errors that yield the same error syndrome? 

**C)** The toric code is a QEC code which topologically protects the logical qubit state. By topological protection, we mean that a logical error cannot occur unless a global property of the system changes. In other words, local noise (like uncorrelated multi-qubit Pauli errors) cannot cause a logical error. Take some time to learn about the toric code on your own:

https://en.wikipedia.org/wiki/Toric_code

https://topocondmat.org/w12_manybody/topoorder.html

<div/>